<a href="https://colab.research.google.com/github/appletreeleaf/Project/blob/main/%EB%8F%84%EC%8B%9C%20%ED%99%98%EA%B2%BD%EC%9A%94%EC%9D%B8%EC%97%90%20%EB%94%B0%EB%A5%B8%20%EB%B2%94%EC%A3%84%EC%9C%A8%20%EC%98%88%EC%B8%A1%EB%AA%A8%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 / 패키지 불러오기

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model



## 1. 데이터 불러오기

In [ ]:
crime = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/AI_start/기말프로젝트/crime (2021 in korea) 수정전.csv", encoding = 'CP949')
crime.tail(10)

### 데이터 시각화

In [ ]:
# 범죄발생건 수( Y값 )의 분포
plt.title('Original Occurrence Histogram')
plt.xticks(rotation=15)
sns.histplot(crime['n_crime_occurrence'], kde=True)
plt.show()

In [ ]:
# 범죄발생건 수( Y값 ) 로그 변환후의 분포
plt.title('Log Transformed Occurrence Histogram')
log_SalePrice = np.log1p(crime['n_crime_occurrence'])
sns.histplot(log_SalePrice, kde=True)
plt.show()

In [ ]:
# 3개의 행과 4개의 열을 가진 subplots를 이용. axs는 4x3개의 ax를 가짐.
fig, axs = plt.subplots(figsize=(16,8) , ncols=4 , nrows=3) #시각화를 위한 빈종이 생성
plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=0.35)
lm_features = ['n_cctv','n_police','n_substation','n_streetlamp','n_emschool','n_119SC','n_bar','n_park', 'n_SC', 'n_underpass', 'population'] #feature를 빈종이로 생성
for i , feature in enumerate(lm_features): #enumerate 인덱스와 인자를 차례로 반환
    row = int(i/4) # 0 0 0 0 1 1 1 1 2 2 2 2
    col = i%4 # 1 2 3 4 1 2 3 4 1 2 3 4
    # 시본의 regplot을 이용해 산점도와 선형 회귀 직선을 함께 표현
    sns.regplot(x=feature , y='n_crime_occurrence',data=crime , ax=axs[row][col]) #row, col 순서에 따른 4*2 그림 출력

# 2. Preprocessing

In [ ]:
# #범죄발생건 수( Y값 ) 로그 변환 (y값의 분포를 정규분포에 근사)

# original_n_crime_occurrence = crime['n_crime_occurrence']

# Dataset에서 X, y 분리
y_target = np.log1p(crime['n_crime_occurrence'])

X_features = crime.drop('n_crime_occurrence', axis=1, inplace=False)
X_features = X_features.drop(['district'], axis=1, inplace=False)  #자치구 칼럼 drop

# 피처 스케일링 -> X들간에 scale 차이를 줄이기 위함
from sklearn.preprocessing import MinMaxScaler

# features 정규화
scaler = MinMaxScaler()
X_features_scaled = scaler.fit_transform(X_features)

## X(정규화) y(로그변환) 산점도

In [ ]:
# 3개의 행과 4개의 열을 가진 subplots를 이용. axs는 4x3개의 ax를 가짐.
fig, axs = plt.subplots(figsize=(16,8) , ncols=4 , nrows=3) #시각화를 위한 빈종이 생성
plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=0.35)
lm_features = ['n_cctv','n_police','n_substation','n_streetlamp','n_emschool','n_119SC','n_bar','n_park', 'n_SC', 'n_underpass', 'population'] #feature를 빈종이로 생성
X_scaled = pd.DataFrame(scaler.fit_transform(X_features), columns=lm_features)
y_log = np.log1p(crime['n_crime_occurrence'])
crime_scaled = pd.concat([X_scaled, y_log], axis = 1) # X, y 데이터프레임 열병='

for i , feature in enumerate(lm_features): #enumerate 인덱스와 인자를 차례로 반환
    row = int(i/4) # 0 0 0 0 1 1 1 1 2 2 2 2
    col = i%4 # 1 2 3 4 1 2 3 4 1 2 3 4
    # 시본의 regplot을 이용해 산점도와 선형 회귀 직선을 함께 표현
    sns.regplot(x=feature , y='n_crime_occurrence',data=crime_scaled , ax=axs[row][col]) #row, col 순서에 따른 4*2 그림 출력

In [ ]:
# train / test 분리
X_train, X_test, y_train, y_test = train_test_split(X_features_scaled, y_target, test_size=0.25, random_state=156)

# 3. 모델 생성

In [ ]:
# 모델 학습
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X_train, y_train, )

In [ ]:
# 예측 (log-scale)
y_pred = mlr.predict(X_test)
y_pred

## 실제 값( target )과 예측값( predict ) 사이의 관계 (산점도)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred, alpha=0.4)
plt.xlabel("Actual Occurance")
plt.ylabel("Predicted Occurance")
plt.title("MULTIPLE LINEAR REGRESSION")
plt.show()

In [ ]:
mlr.score(X_train, y_train) # R^2 (결정계수 / accuracy와 같은 역할)

# 선형 회귀 모델의 학습/예측/평가
* RMSE 평가 함수 생성
타겟값을 로그 변환하여, 학습한 모델은 예측값도 원본 데이터에 로그 스케일로 예측함.
실제 타겟값도 로그값, 예측값도 로그값이므로 RMSE를 적용하면 자연스럽게 RMSLE로 적용됨.

In [ ]:
# 학습이 완료된 모델을 인자로 받아서 테스트 데이터로 예측하고 RMSE를 계산
def get_rmse(model):
    pred = model.predict(X_test)
    # y_test, pred는 로그 스케일임.
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

# 여러 모델들을 list 형태로 인자로 받아서 개별 모델들의 RMSE를 list로 반환.
def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

**LinearRegression, Ridge, Lasso 학습, 예측, 평가**

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# LinearRegression, Ridge, Lasso 학습, 예측, 평가
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha=20)
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso(alpha=0.05)
lasso_reg.fit(X_train, y_train)

models = [lr_reg, ridge_reg, lasso_reg]
print(get_rmses(models))


**회귀 계수값과 컬럼명 시각화를 위해 상위 10개, 하위 10개(-값으로 가장 큰 10개) 회귀 계수값과 컬럼명을 가지는 Series생성 함수.**

In [ ]:
coef = pd.Series(lasso_reg.coef_, index=X_features.columns)

# + 상위 10개 , - 하위 10개 coefficient 추출하여 반환.
coef_high = coef.sort_values(ascending=False).head(3)
coef_low = coef.sort_values(ascending=False).tail(3)
print(coef_high)
print('---------------------------------')
print(coef_low)

In [ ]:
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):
    for model in models:
        # 분할하지 않고 전체 데이터로 cross_val_score( ) 수행. 모델별 CV RMSE값과 평균 RMSE 출력
        rmse_list = np.sqrt(-cross_val_score(model, X_features, y_target,
                                             scoring="neg_mean_squared_error", cv = 5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0} CV RMSE 값 리스트: {1}'.format( model.__class__.__name__, np.round(rmse_list, 3)))
        print('{0} CV 평균 RMSE 값: {1}'.format( model.__class__.__name__, np.round(rmse_avg, 3)))

# 앞 예제에서 학습한 ridge_reg, lasso_reg 모델의 CV RMSE값 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_avg_rmse_cv(models)

In [ ]:
from sklearn.model_selection import GridSearchCV

def print_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params,
                              scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_features, y_target)
    rmse = np.sqrt(-1* grid_model.best_score_)
    print('{0} 5 CV 시 최적 평균 RMSE 값: {1}, 최적 alpha:{2}'.format(model.__class__.__name__,
                                        np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

ridge_params = { 'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20] }
lasso_params = { 'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1,5, 10] }
best_rige = print_best_params(ridge_reg, ridge_params)
best_lasso = print_best_params(lasso_reg, lasso_params)


In [ ]:
lasso_reg.score(X_train, y_train) # R-squared (결정계수) -> 학습데이터가 총변동을을 설명해줌.

In [ ]:
X_train, y_train

# 번외) Keras를 이용한 다중선형회귀 모형


Keras 모델 생성 및 학습

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_dim = 11, activation = 'linear'))
model.compile(optimizer = 'adam', loss='mse')
model.fit(X_train, y_train, batch_size=1, epochs=200)

In [ ]:
predictions = model.predict(X_test)
test_loss = model.evaluate(X_test, y_test)
print( "Weights = ", model.get_weights() )

In [ ]:
pip install nbconvert

In [ ]:
jupyter nbconvert --to markdown 인공지능_GBT_기말프로젝트_1128.ipynb